# Imports

In [1]:
from matplotlib.patches import Rectangle
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')

# Data

In [2]:
data = pd.read_csv('../data/model-pitches-rv.csv', index_col = [0])
data.drop(columns = ['plate_x', 'release_pos_x'], inplace = True)
data.rename(columns = {'pfx_-x': 'pfx_x', 
                       'plate_-x': 'plate_x', 
                       'release_pos_-x': 'release_pos_x'}, 
            inplace = True)

rv_pitches = pd.read_csv('../data/pitches-rv-table.csv', index_col = [0])
grouped_pitches_rv = pd.read_csv('../data/grouped-pitches-rv-table.csv', index_col = [0])

pd.set_option('max_columns', None)
print(data.shape)
data.head()

(705396, 70)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_x,pfx_z,bauer_units,effective_speed,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_x,plate_z,type,balls,strikes,pitch_count,delta_run_exp,stand,bb_type,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,woba_value,woba_denom,xba,xwoba,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,is_strike,is_ball,final_pitch_ab,out_to_end_inning,home_runs,away_runs,runs,re,re_change,re_end_state,re24,lin_weight_above_avg,lin_weight_above_outs,woba_scale,lin_weights_above_avg_scale,lin_weights_above_outs_scale,woba,wraa_change,count_re,bs_lin_weight,bs_lin_weight_scale,rv_above_avg,rv
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,-1.40,6.80,6.5,54.03,0.69,2.83,X,1,2,1-2,-0.073,R,ground_ball,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.0,1.0,0.174,0.158,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.302,0.0,0.223,-0.184,-0.282,-0.141,-0.170,-0.472,-0.170
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,-1.60,6.64,6.4,54.15,0.71,2.62,S,1,1,1-1,-0.027,R,NaN,strike,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,0,5,0,5,0,0,0,0,2,1,0,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.293,-0.058,-0.058,-0.006,-0.007,-0.007,-0.007
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,-1.46,6.88,6.2,54.34,0.04,2.46,S,1,0,1-0,-0.020,R,NaN,strike,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,0,5,0,5,0,0,0,0,2,1,0,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.355,-0.051,-0.051,0.001,0.001,0.001,0.001
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,-1.53,6.83,5.9,54.61,-2.10,3.89,B,0,0,0-0,0.016,R,NaN,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,0,5,0,5,0,0,0,0,2,0,1,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.314,0.034,0.034,-0.003,-0.004,-0.004,-0.004
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,-1.49,6.66,6.3,54.15,0.31,2.80,X,1,0,1-0,-0.189,L,ground_ball,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.0,1.0,0.100,0.090,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.302,0.0,0.355,-0.051,-0.207,0.001,0.001,-0.301,0.001


# Analysis

In [3]:
data.groupby(['events'], as_index = False)['re24', 'lin_weight_above_avg', 'lin_weight_above_outs'].mean()

,events,re24,lin_weight_above_avg,lin_weight_above_outs
0,double,0.765841,0.766,1.016
1,field_error,0.427661,0.428,0.678
2,field_out,-0.249793,-0.250,0.000
3,hit_by_pitch,0.340325,0.340,0.590
4,home_run,1.394975,1.395,1.645
5,single,0.457643,0.458,0.708
6,triple,1.052801,1.053,1.303
7,walk,0.313550,0.314,0.564


In [4]:
data.groupby(['events'], as_index = False)['rv_above_avg', 'rv'].mean()

,events,rv_above_avg,rv
0,double,0.863998,1.165998
1,field_error,0.444663,0.747663
2,field_out,-0.405923,-0.103923
3,hit_by_pitch,0.335008,0.637008
4,home_run,1.632701,1.934701
5,single,0.486939,0.788939
6,triple,1.206801,1.508801
7,walk,0.338175,0.640175


In [5]:
data.groupby(['pitch_count'], as_index = False)['woba', 'wraa_change', 'count_re', 'bs_lin_weight'].mean()

,pitch_count,woba,wraa_change,count_re,bs_lin_weight
0,0-0,0.314,-0.012715,-0.003127,-0.003
1,0-1,0.262,-0.022642,-0.013365,-0.013
2,0-2,0.196,-0.080462,-0.123197,-0.123
3,1-0,0.355,-0.010147,0.001278,0.001
4,1-1,0.293,-0.020787,-0.005869,-0.006
...,...,...,...,...,...
7,2-1,0.352,-0.016841,0.000218,0.000
8,2-2,0.273,-0.129371,-0.168415,-0.168
9,3-0,0.622,-0.064147,0.053291,0.053
10,3-1,0.470,0.001302,0.115239,0.115


In [6]:
data.groupby(['pitch_count'], as_index = False)['woba', 'wraa_change', 'bs_lin_weight', 'rv'].mean()

,pitch_count,woba,wraa_change,bs_lin_weight,rv
0,0-0,0.314,-0.012715,-0.003,0.038423
1,0-1,0.262,-0.022642,-0.013,0.045727
2,0-2,0.196,-0.080462,-0.123,-0.087995
3,1-0,0.355,-0.010147,0.001,0.059814
4,1-1,0.293,-0.020787,-0.006,0.068386
...,...,...,...,...,...
7,2-1,0.352,-0.016841,0.000,0.085070
8,2-2,0.273,-0.129371,-0.168,-0.114041
9,3-0,0.622,-0.064147,0.053,0.314586
10,3-1,0.470,0.001302,0.115,0.425531


In [7]:
data.groupby(['player_name', 'pitch_type'], as_index = False)['rv_above_avg', 'rv'].sum()

,player_name,pitch_type,rv_above_avg,rv
0,"Abad, Fernando",CH,-4.676,-1.052
1,"Abad, Fernando",FC,-0.730,0.780
2,"Abad, Fernando",FF,1.025,2.837
3,"Abad, Fernando",KC,-2.988,3.354
4,"Abad, Fernando",SI,-3.458,7.717
...,...,...,...,...
3464,"Álvarez, José",CH,-21.505,-0.968
3465,"Álvarez, José",FC,-1.746,-0.840
3466,"Álvarez, José",FF,-7.891,1.774
3467,"Álvarez, José",SI,-29.104,8.949


# Preprocessing

### Multicolinearity - VIF

**Independent Variables:** Velocity, Spin Rate, VB, HB, Release Extension, Horizontal Release Position, Vertical Release Position, Horizontal Plate Coords, Vertical Plate Coords

**Dependent Variable:** rv

In [8]:
features = data[['player_name', 'p_throws', 'pitch_type', 'velo', 'spin_rate', 
                 'pfx_x', 'pfx_z', 'release_pos_x', 'release_pos_z', 'release_extension',
                 'plate_x', 'plate_z', 'rv']]

features_vif = features.select_dtypes([np.number])
vif_data = pd.DataFrame()
vif_data["feature"] = features_vif.columns

vif_data["VIF"] = [variance_inflation_factor(features_vif.values, i)
                   for i in range(len(features_vif.columns))]

vif_data.sort_values(by = 'VIF').head(10)

,feature,VIF
9,rv,1.012208
7,plate_x,1.110543
4,release_pos_x,1.454762
2,pfx_x,1.466135
3,pfx_z,3.114446
8,plate_z,7.451706
1,spin_rate,50.839720
5,release_pos_z,103.361778
6,release_extension,152.722320
0,velo,277.759432
